In [ ]:
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams
from langchain_google_genai import GoogleGenerativeAIEmbeddings

import os
from dotenv import load_dotenv
load_dotenv()

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")


qdrant_api_key = os.getenv("QDRANT_API_KEY")
client = QdrantClient(path="qdrant_data")

# client = QdrantClient(
#     url="https://b5e3e25c-0644-477b-8c25-76b8e3c4fb7a.us-east-1-0.aws.cloud.qdrant.io:6333", 
#     api_key=qdrant_api_key,
# )

client.create_collection(
    collection_name="demo_collection",
    vectors_config=VectorParams(size=768, distance=Distance.COSINE),
)

vector_store = QdrantVectorStore(
    client=client,
    collection_name="demo_collection",
    embedding=embeddings,
)

### Save embedding to Qdrant

In [ ]:
from langchain_community.document_loaders import UnstructuredMarkdownLoader
from langchain.text_splitter import MarkdownHeaderTextSplitter
from langchain_community.document_loaders import TextLoader

def load_split(file_path):
    # Load Markdown file
    loader = TextLoader(file_path, encoding="utf-8")
    # loader = UnstructuredMarkdownLoader(file_path)
    documents = loader.load()

    # Extract raw text from the first Document object
    markdown_text = documents[0].page_content  # ✅ Use page_content string

    # Markdown splitter by headers (e.g., "# Page")
    markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=[("##", "page")])
    docs = markdown_splitter.split_text(markdown_text)  # ✅ Now it's a string

    return docs


In [ ]:
file_path = "./knowledge/combined.md"
chunks = load_split(file_path)
vector_store.add_documents(documents=chunks)


In [ ]:
def response_generator(vectordb, query, llm):
    context = vectordb.similarity_search(query, k=4)

    # for res in results:
    #     print(f"📄 {res.page_content} [{res.metadata}]")
    template = f"""
    You are an intelligent assistant designed to provide accurate and concise answers based on the context provided. 
    Follow these rules strictly:
    1. Use ONLY the information provided in the context to answer the question.
    2. If the context does not contain enough information to answer the question, say "I don't know."
    3. Do not make up or assume any information outside of the context.
    4. Keep your answer concise and to the point (maximum 3 sentences).

    Context:
    {context}

    Question:
    {query}

    Helpful Answer:
    """
    ans = llm.invoke(template)
    return ans.content

In [ ]:

def response_generatorr(vectordb, query, llm):
    # vectordb = get_vectordb()
    # llm = get_llm()
    context = vectordb.similarity_search(query, k=4)

    # for res in results:
    #     print(f"📄 {res.page_content} [{res.metadata}]")
    template = f"""
    You are an intelligent assistant designed to provide accurate and concise answers based on the context provided. 
    Follow these rules strictly:
    1. Use ONLY the information provided in the context to answer the question.
    2. If the context does not contain enough information to answer the question, say "I don't know."
    3. Do not make up or assume any information outside of the context.
    4. Keep your answer concise and to the point (maximum 3 sentences).

    Context:
    {context}

    Question:
    {query}

    Helpful Answer:
    """
    ans = llm.invoke(template)
    return ans.content